# Si446x Device Direct Access Ping

In [ ]:
%autosave 0
import sys
sys.path.append("../") # go to parent dir
%run si446x_Device_Layer.ipynb
#test

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))
print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
radio = si446x_device_start_radio()

In [ ]:
si446x_device_show_config(radio.dump_radio())

## Check for Command Error

In [ ]:
status = radio.get_chip_status()
if (status.chip_pend.CMD_ERROR):
    print(status)

##  Configure Radio

In [ ]:
config = si446x_device_config_radio(radio)

si446x_device_show_config(radio.dump_radio())
total = 0
print('\n=== const config strings:')
for s in config:
    print((hexlify(s)))
    total += len(s) - 4
print('\n total: {}'.format(total))

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
from ipywidgets import interact
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
f = True
status = int_status(0, f)

## Transmit Packet Stream

In [ ]:
from __future__ import print_function
from time import sleep
from datetime import datetime

num         = 10000
delay       = .5
pkt         = bytearray(256)

step        = 1
prp         = bytearray(1)
mj_r        = range(num)
th_r        = range(25,46,5)             # fifo threshold setting (min,max,opt): 1,64,40
sz_r        = range(250,260,10)             # msg size (min,max): 4, 255
pw_r        = [1 << i for i in range(1)] # power amp setting (min, max,opt): 1,127,10
count       = 0
radio.trace._enable()

start = datetime.now()
print(start)

for major in mj_r:
    for thresh in th_r:
        prp[0]=thresh
        radio.set_property('PKT', 0x0b, prp)  # set tx fifo threshhold
        for pw in pw_r:
            if (pw > 127): pw = 127
            for size in sz_r:
                if (size > 255): size = 254
                for i in range(1,len(pkt)):
                    pkt[i] = i
                pkt[i] = size
                progress = x_si446x_device_send_msg(radio, pkt[:size], pw)
                count += 1
                print('\r{}, c:{}, l:{}, p:{}, t:{}, m:{}, s:{}'.format(
                    datetime.now(), count, size, pw, thresh, hexlify(pkt[:4]), 
                    ''.join(map(str, progress))),end='')
                if (progress[-1] == 'e'): print()
                else: print('                    ',end='')
                sleep(delay)

print('\ndone')

## Check Interrupt Status

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
s_name =  'get_chip_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_chip_status()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
#t,r = si446xradio_test()

In [ ]:
from datetime import datetime 

now=datetime.now()
print(now)

In [ ]:
MAX_CHUNK=64

In [ ]:
radio.clear_interrupts()

In [ ]:
rx, tx = radio.fifo_info(rx_flush=True, tx_flush=True)
print(rx,tx)

In [ ]:
radio.set_property('PKT', 0x0b, b'\x3c')  # set tx fifo threshhold

In [ ]:
rx, tx = radio.fifo_info()
print(rx,tx)

In [ ]:
status = radio.get_interrupts()
print(status.ph_pend.TX_FIFO_ALMOST_EMPTY)

In [ ]:
s_name =  'int_status_rsp_s'
p_s = eval(s_name)
p_g = radio.get_interrupts()
p_d = p_s.build(p_g)
print('{}: {}'.format(s_name, insert_space(p_d)))
print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
buf = bytearray('\x22' * 64)
chunk = 8
cur   = 8
radio.write_tx_fifo(buf[:chunk])
rx, tx = radio.fifo_info()
print(rx,tx)

In [ ]:
radio.start_tx(len(buf))
while (True):
    status = radio.get_interrupts()
    if (status.ph_pend.TX_FIFO_ALMOST_EMPTY):
        rx, tx = radio.fifo_info()
        radio.write_tx_fifo(buf[cur:cur+chunk])
        cur += chunk
        print('.',end='')
    if (status.ph_pend.PACKET_SENT):
        print('done')
        break
    radio.clear_interrupts()

In [ ]:
ss[0] = 10
ss[1] = 20

In [ ]:
cur

In [ ]:
radio.set_property('PKT', 0x0b, ss) # tx/rx threshold

In [ ]:
radio.set_property('PKT', 0x0b, b'\x48')  # set tx fifo threshhold

In [ ]:
radio.set_property('PKT', 0x0c, b'\x14')  # set rx fifo threshold

In [ ]:
status = int_status(clr_all_flags, False)
print(status.ph_pend.TX_FIFO_ALMOST_EMPTY)